In [1]:
import json
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier  

In [6]:
# Read csv
df = pd.read_csv('../../datasets/ASG_train.csv') 
# all_stars = all_players.loc[all_players['Selected?'] == 1]
df.head(10)

Year  Avg. Pace            PLAYER TEAM  Team Conference Rank  GP  Team GP  \
0  1996       90.1    Michael Jordan  CHI                     1  40       40   
1  1996       90.1  Shaquille O'Neal  LAL                     3  40       41   
2  1996       90.1  Latrell Sprewell  GSW                     7  39       39   
3  1996       90.1       Karl Malone  UTA                     4  40       40   
4  1996       90.1   Hakeem Olajuwon  HOU                     1  37       41   
5  1996       90.1    Mitch Richmond  SAC                     8  40       40   
6  1996       90.1         Glen Rice  CHH                     6  37       40   
7  1996       90.1     Allen Iverson  PHI                    15  34       39   
8  1996       90.1         Vin Baker  MIL                    10  36       39   
9  1996       90.1       Gary Payton  SEA                     2  40       40   

    W   PTS   REB  ...  BLK  TOV   TS%  3PM  DEFWS  USG%   PIE  \
0  35  30.9   5.8  ...  0.5  1.7  56.4  1.2  0.004  33.6  19.8   
1  28  26.2  13.2  ...  3.1  3.1  55.7  0.0  0.003  30.4  18.4   
2  16  25.9   4.9  ...  0.8  4.0  57.1  2.2  0.001  28.2  14.5   
3  27  25.8  10.8  ...  0.7  3.1  57.6  0.0  0.003  31.5  20.8   
4  28  24.1   9.4  ...  2.2  3.7  54.8  0.1  0.003  32.2  16.3   
5  16  24.1   3.9  ...  0.3  2.8  56.0  1.8  0.002  28.4  14.3   
6  21  24.0   4.2  ...  0.4  2.2  57.8  2.5  0.002  25.4  11.3   
7   7  22.8   4.5  ...  0.4  4.9  50.3  2.2  0.000  29.0  10.3   
8  18  22.4  10.7  ...  1.4  3.6  56.2  0.2  0.003  26.9  14.2   
9  29  22.2   4.4  ...  0.1  2.5  55.1  1.4  0.004  26.0  15.6   

   Prior ASG Appearances  AS Last Year?  Selected?  
0                     10              1          1  
1                      4              1          1  
2                      2              0          1  
3                      9              1          1  
4                     11              1          1  
5                      4              1          1  
6                      1              1          1  
7                      0              0          0  
8                      2              1          1  
9                      3              1          1  

[10 rows x 22 columns]

In [25]:
label_name = 'Selected?'

'Selected?'

In [20]:
features = (all_players.drop(['Selected?', 'Year', 'Avg. Pace', 'PLAYER', 'TEAM', 'AS Last Year?'], axis=1).columns.values)

In [21]:
df_dummy = all_players.drop(['Selected?', 'Year', 'Avg. Pace', 'PLAYER', 'TEAM', 'AS Last Year?'], axis=1)

In [22]:
def generator_1(clf, features, labels,original_features, node_index=0,side=0):
  
    node = {}
    if clf.tree_.children_left[node_index] == -1:  # indicates leaf
        count_labels = zip(clf.tree_.value[node_index, 0], labels)
        node['name'] = ', '.join(('{} of {}'.format(int(count), label)
                                  for count, label in count_labels))
        node['size'] = sum( clf.tree_.value[node_index, 0]  )   
        node['side'] = 'left' if side == 'l' else 'right'                      
    else:

        count_labels = zip(clf.tree_.value[node_index, 0], labels)
        node['pred'] = ', '.join(('{} of {}'.format(int(count), label)
                                  for count, label in count_labels))
                                      
        node['side'] = 'left' if side == 'l' else 'right'                              
        feature = features[clf.tree_.feature[node_index]]
        threshold = clf.tree_.threshold[node_index]
        
        if ('_-_' in feature) and (feature not in original_features):
            node['name'] =  '{} = {}'.format(feature.split('_-_')[0], feature.split('_-_')[1] )
            node['type'] = 'categorical'
        else:
            node['name'] = '{} > {}'.format(feature, round(threshold,2) )
            node['type'] = 'numerical'
        
        left_index = clf.tree_.children_left[node_index]
        right_index = clf.tree_.children_right[node_index]
        
        node['size'] = sum (clf.tree_.value[node_index, 0])
        node['children'] = [generator_1(clf, features, labels, original_features, right_index,'r'),
                            generator_1(clf, features, labels, original_features, left_index,'l')]
                            
        
    return node



In [23]:
def generator_2(clf, features, labels,original_features, node_index=0,side=0,prev_index=0):

    node = {}
    if clf.tree_.children_left[node_index] == -1:  # indicates leaf
        count_labels = zip(clf.tree_.value[node_index, 0], labels)
        node['pred'] = ', '.join(('{} of {}'.format(int(count), label)
                                  for count, label in count_labels))
                                      
        node['side'] = 'left' if side == 'l' else 'right'                              
        feature = features[clf.tree_.feature[prev_index]]
        threshold = clf.tree_.threshold[prev_index]
        
            
        if node_index == 0:
            node["name"] = 'Root >'
        elif ('_-_' in feature) and (feature not in original_features):
            
            node['name'] =  '{} = {}'.format(feature.split('_-_')[0], feature.split('_-_')[1] ) if side == 'r' else '{} != {}'.format(feature.split('_-_')[0], feature.split('_-_')[1] )  
            node['type'] = 'categorical'
        else:
            node['name'] = '{} > {}'.format(feature, round(threshold,2) ) if side == 'r' else '{} <= {}'.format(feature, round(threshold,2) ) 
            node['type'] = 'numerical'
        
        left_index = clf.tree_.children_left[node_index]
        right_index = clf.tree_.children_right[node_index]
        
        node['size'] = sum (clf.tree_.value[node_index, 0])
           
    else:

        count_labels = zip(clf.tree_.value[node_index, 0], labels)
        node['pred'] = ', '.join(('{} of {}'.format(int(count), label)
                                  for count, label in count_labels))
                                      
        node['side'] = 'left' if side == 'l' else 'right'                              
        feature = features[clf.tree_.feature[prev_index]]
        threshold = clf.tree_.threshold[prev_index]
        
            
        if node_index == 0:
            node["name"] = 'Root >'
        elif ('_-_' in feature) and (feature not in original_features):
            
            node['name'] =  '{} = {}'.format(feature.split('_-_')[0], feature.split('_-_')[1] ) if side == 'r' else '{} != {}'.format(feature.split('_-_')[0], feature.split('_-_')[1] )  
            node['type'] = 'categorical'
        else:
            node['name'] = '{} > {}'.format(feature, round(threshold,2) ) if side == 'r' else '{} <= {}'.format(feature, round(threshold,2) ) 
            node['type'] = 'numerical'
        
        left_index = clf.tree_.children_left[node_index]
        right_index = clf.tree_.children_right[node_index]
        
        node['size'] = sum (clf.tree_.value[node_index, 0])
        node['children'] = [generator_2(clf, features, labels, original_features, right_index,'r',node_index),
                            generator_2(clf, features, labels, original_features, left_index,'l',node_index)]
                            
        
    return node
    
    


In [26]:
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(df_dummy, df[label_name])

io=generator_1(clf, df_dummy.columns,np.unique(df[label_name]),features)

print(json.dumps(io, indent=4))

with open('structureC1.json', 'w') as outfile:
    json.dump(io, outfile, indent=4)
    
    
io=generator_2(clf, df_dummy.columns,np.unique(df[label_name]),features)

print(json.dumps(io, indent=4))

with open('structureC2.json', 'w') as outfile:
    json.dump(io, outfile, indent=4)

{
    "pred": "9025 of 0, 570 of 1",
    "side": "right",
    "name": "PTS > 18.45",
    "type": "numerical",
    "size": 9595.0,
    "children": [
        {
            "pred": "302 of 0, 420 of 1",
            "side": "right",
            "name": "PIE > 14.25",
            "type": "numerical",
            "size": 722.0,
            "children": [
                {
                    "pred": "61 of 0, 306 of 1",
                    "side": "right",
                    "name": "W > 19.5",
                    "type": "numerical",
                    "size": 367.0,
                    "children": [
                        {
                            "pred": "18 of 0, 239 of 1",
                            "side": "right",
                            "name": "PTS > 22.05",
                            "type": "numerical",
                            "size": 257.0,
                            "children": [
                                {
                                    "pred": "1 of